In [11]:
source("../00_model-fitting/wwa_model-fitting.R")

# **Linear trend in log10_cumDSR**

## **log10_cumDSR per HFRZ**

### **Prep data**

In [16]:
# load data
gmst <- read.fwf("/home/clair/Data/gmst.dat", comment.char = "#", widths = c(5,15), col.names = c("year", "gmst"))
cdsr <- read.csv("data/era5_cdsr.csv")

# take base10 log of DSR as in original study
log10_cdsr <- sapply(colnames(cdsr)[-1], function(cnm) log10(cdsr[,cnm]))
colnames(log10_cdsr) <- paste0("log10_", colnames(log10_cdsr))       
log10_cdsr <- data.frame("year" = cdsr$year, log10_cdsr)
                     
# merge into single dataframe
df <- merge(gmst, log10_cdsr)

### **Check distribution of log10_cumDSR**

In [3]:
prep_window(c(4,4))
x <- seq(0.6,3,0.01)

jpeg("fig/expl_era5_cdsr-fit.jpg", width = 480 * 3, height = 480 * 2); par(mfrow = c(4,4), lwd = 2, cex = 1.2, mar = c(3,3,2,1)); {
    invisible(sapply(colnames(df)[-(1:2)], function(cnm) { 
        ts <- df[,cnm]
        hist(ts, prob = T, xlim = c(0.8,3), ylim = c(0,3), main = cnm, xlab = "", ylab = "")
        lines(density(ts))
        lines(x, dnorm(x, mean(ts), sd(ts)), col = "blue")
    }))
}; dev.off()

png 
  2

### **Fit linear trend model**

In [17]:
models <- sapply(colnames(df)[-(1:2)], function(cnm) fit_ns("norm", "shift", df, cnm, "gmst", lower = F), simplify = F)
gmst_2023 <- gmst[gmst$year == 2023, "gmst"]
gmst_hist <- gmst_2023 - 1.2

In [4]:
res <- sapply(models, function(mdl) mdl_ests(mdl, cov = gmst_2023, cov_cf = gmst_hist, ev = mdl$ev))

In [10]:
res[c("mu0", "alpha", "sigma0"),]

,log10_EJB,log10_ES,log10_ET,log10_GBL,log10_GSL,log10_IC,log10_LA,log10_LW,log10_NAT,log10_P,log10_SC,log10_SP,log10_SY,log10_WJB,log10_WO,log10_WS
mu0,1.7706601,1.4852844,2.0648489,2.0689752,2.48391102,2.08079934,2.2010444,2.10729064,1.68624332,1.78207481,2.3616798,2.534200256,1.9098785,2.1259144,1.9871093,2.0686211
alpha,0.1844718,0.2715079,0.0705635,0.2186396,0.06273564,0.07145091,0.1209055,0.08404658,0.03979895,0.09941973,0.1342584,-0.005470569,0.1175365,0.1126802,0.1153882,0.1423676
sigma0,0.1727815,0.2372574,0.1458640,0.1663821,0.15573033,0.18842426,0.1744042,0.16007508,0.14217505,0.18291477,0.1579131,0.165353315,0.2026454,0.1481913,0.1797439,0.1810322


In [6]:
write.csv(res, file = "res/cdsr_lineartrend-ests.csv")

In [7]:
prep_window(c(4,4))

jpeg("fig/expl_era5_cdsr-trend.jpg", width = 480 * 3, height = 480 * 2); par(mfrow = c(4,4), lwd = 2, cex = 1.2, mar = c(3,3,2,1)); {
    invisible(lapply(models, function(mdl) { 
        plot_trend(mdl, legend_pos = NA, main = mdl$varnm)
        lines(df$year, filter(mdl$x, rep(1/15,15), sides = 2), col = "forestgreen", lwd = 2)
    }))
}; dev.off()

png 
  2

In [8]:
prep_window(c(4,4))

jpeg("fig/expl_era5_cdsr-gmsttrend.jpg", width = 480 * 3, height = 480 * 2); par(mfrow = c(4,4), lwd = 2, cex = 1.2, mar = c(3,3,2,1)); {
    invisible(lapply(models, function(mdl) { 
        plot_gmsttrend(mdl, legend_pos = NA, main = mdl$varnm, cov = gmst_2023, cov_cf = gmst_hist)
        lines(df$gmst[order(df$gmst)], filter(mdl$x[order(df$gmst)], rep(1/15,15), sides = 2), col = "forestgreen", lwd = 2)
    }))
}; dev.off()

png 
  2

In [9]:
prep_window(c(4,4))

jpeg("fig/expl_era5_cdsr-rlplots.jpg", width = 480 * 3, height = 480 * 2); par(mfrow = c(4,4), lwd = 2, cex = 1.2, mar = c(3,3,2,1)); {
    invisible(lapply(models, function(mdl) { 
        plot_returnlevels(mdl, legend_pos = NA, main = mdl$varnm, cov = gmst_2023, cov_cf = gmst_hist)
    }))
}; dev.off()

### **Estimated return periods**

In [18]:
models <- sapply(colnames(df)[-(1:2)], function(cnm) fit_ns("norm", "shift", df, cnm, "gmst", lower = F), simplify = F)
gmst_2023 <- gmst[gmst$year == 2023, "gmst"]
gmst_hist <- gmst_2023 - 1.2

In [25]:
boots <- lapply(models, function(mdl) boot_ci(mdl, cov = gmst_2023, cov_cf = gmst_hist, ev = mdl$ev))

In [31]:
round(sapply(boots, "[", "return_period",)[,(1:8)], 2)

,log10_EJB,log10_ES,log10_ET,log10_GBL,log10_GSL,log10_IC,log10_LA,log10_LW
bestimate,41.64,20.01,4.17,8.28,43.41,11.38,11.06,2.99
2.5%,14.31,7.56,2.63,3.93,14.68,5.82,5.59,2.09
97.5%,195.12,74.30,7.84,22.78,237.70,31.87,27.81,5.01


In [30]:
round(sapply(boots, "[", "return_period",)[,(9:16)], 2)

,log10_NAT,log10_P,log10_SC,log10_SP,log10_SY,log10_WJB,log10_WO,log10_WS
bestimate,1.58,12.04,3.03,12.52,3.06,75.80,5.85,4.85
2.5%,1.25,5.97,2.02,6.76,1.92,24.53,3.43,2.78
97.5%,2.13,39.77,5.57,31.70,6.52,547.92,12.72,10.13


### **Change in intensity**

In [33]:
round(sapply(boots, "[", "dI_rel",)[,(1:8)], 2)

,log10_EJB,log10_ES,log10_ET,log10_GBL,log10_GSL,log10_IC,log10_LA,log10_LW
bestimate,66.48,111.74,21.53,82.96,18.93,21.83,39.66,26.14
2.5%,21.47,36.57,-3.82,32.53,-8.64,-12.90,3.91,-4.69
97.5%,122.84,219.30,49.74,155.30,62.15,69.94,94.42,62.81


In [34]:
round(sapply(boots, "[", "dI_rel",)[,(9:16)], 2)

,log10_NAT,log10_P,log10_SC,log10_SP,log10_SY,log10_WJB,log10_WO,log10_WS
bestimate,11.62,31.61,44.91,-1.50,38.37,36.53,37.55,48.20
2.5%,-11.73,-3.82,9.63,-24.41,-7.73,4.68,3.75,10.45
97.5%,47.07,86.12,94.10,28.11,102.05,75.41,83.69,107.53


### **Probability ratio**

In [36]:
round(sapply(boots, "[", "PR",)[,(1:8)], 2)

,log10_EJB,log10_ES,log10_ET,log10_GBL,log10_GSL,log10_IC,log10_LA,log10_LW
bestimate,42.85,39.32,2.42,40.27,3.49,2.49,6.03,2.31
2.5%,3.88,4.24,0.84,4.50,0.47,0.50,1.22,0.85
97.5%,633.28,642.82,6.79,781.92,40.65,14.55,57.99,6.93


In [37]:
round(sapply(boots, "[", "PR",)[,(9:16)], 2)

,log10_NAT,log10_P,log10_SC,log10_SP,log10_SY,log10_WJB,log10_WO,log10_WS
bestimate,1.26,3.99,4.58,0.93,2.59,15.25,4.01,5.30
2.5%,0.76,0.81,1.39,0.25,0.78,1.58,1.15,1.48
97.5%,2.24,25.66,22.41,3.60,9.17,201.81,17.92,30.40


## **log10_cumDSR over all HFRZs**

### **Prep data**

In [20]:
# load data
gmst <- read.fwf("/home/clair/Data/gmst.dat", comment.char = "#", widths = c(5,15), col.names = c("year", "gmst"))
gmst$gmst <- gmst$gmst - gmst[gmst$year == 2023, "gmst"]

cdsr <- read.csv("data/era5_cdsr-canada.csv")
df <- merge(gmst, cdsr)
df$log10_cdsr <- log10(df$cdsr)

### **Linear model**

In [21]:
mdl <- fit_ns("norm", "shift", df, "log10_cdsr", "gmst", lower = F)

gmst_2023 <- gmst[gmst$year == 2023, "gmst"]
gmst_hist <- gmst_2023 - 1.2

In [28]:
prep_window(c(1,3))

jpeg("fig/expl_era5_cdsr-gmsttrend_canada.jpg", height = 480, width = 480*4); par(mfrow = c(1,3), lwd = 2, cex = 1.2, mar = c(3,3,2,1)); {
    
    plot_trend(mdl, main = "Trend in cumDSR over time")
    lines(df$year, filter(mdl$x, rep(1/15,15), sides = 2), col = "forestgreen", lwd = 2)
    
    plot_gmsttrend(mdl, cov = gmst_2023, cov_cf = gmst_hist)
    lines(df$gmst[order(df$gmst)], filter(mdl$x[order(df$gmst)], rep(1/15,15), sides = 2), col = "forestgreen", lwd = 2)
    
    plot_returnlevels(mdl, cov = gmst_2023, cov_cf = gmst_hist)
    
}; dev.off()

png 
  2

In [31]:
t(round(boot_ci(mdl, cov = gmst_2023, cov_cf = gmst_hist, ev = mdl$ev), 3))[,-c(4,5,8)]

,mu0,sigma0,alpha,return_period,PR,dI_rel
bestimate,6.421,0.094,0.085,48.663,22.701,26.363
2.5%,6.380,0.081,0.024,16.618,2.366,6.834
97.5%,6.463,0.107,0.147,212.340,385.090,50.285


---
## **Sensitivity to start of time series (ERA5)**

In [10]:
# load data
gmst <- read.fwf("/home/clair/Data/gmst.dat", comment.char = "#", widths = c(5,15), col.names = c("year", "gmst"))
cdsr <- read.csv("data/era5_cdsr.csv")

# take base10 log of DSR as in original study
log10_cdsr <- sapply(colnames(cdsr)[-1], function(cnm) log10(cdsr[,cnm]))
colnames(log10_cdsr) <- paste0("log10_", colnames(log10_cdsr))       
log10_cdsr <- data.frame("year" = cdsr$year, log10_cdsr)
                     
# merge into single dataframe
df <- merge(gmst, log10_cdsr)
                     
# fit all the models, also excluding years prior to 1950/1960
models_1940 <- sapply(colnames(df)[-(1:2)], function(cnm) fit_ns("norm", "shift", df[df$year >= 1940,], cnm, "gmst", lower = F), simplify = F)
models_1950 <- sapply(colnames(df)[-(1:2)], function(cnm) fit_ns("norm", "shift", df[df$year >= 1950,], cnm, "gmst", lower = F), simplify = F)
models_1960 <- sapply(colnames(df)[-(1:2)], function(cnm) fit_ns("norm", "shift", df[df$year >= 1960,], cnm, "gmst", lower = F), simplify = F)

gmst_2023 <- gmst[gmst$year == 2023, "gmst"]
gmst_hist <- gmst_2023 - 1.2

In [11]:
prep_window(c(4,4))

jpeg("fig/expl-sa_era5_cdsr-trend_start-sensitivity.jpg", width = 480 * 3, height = 480 * 2); par(mfrow = c(4,4), lwd = 2, cex = 1.2, mar = c(3,3,2,1)); {
    invisible(sapply(1:16, function(i) {    
        plot(df$year, df[,-(1:2)][,i], type = "S", main = models_1940[[i]]$varnm)

        lines(models_1940[[i]]$data$year, ns_pars(models_1940[[i]])$loc)
        lines(models_1950[[i]]$data$year, ns_pars(models_1950[[i]])$loc, col = "darkblue")
        lines(models_1960[[i]]$data$year, ns_pars(models_1960[[i]])$loc, col = "darkred")
        
        lines(df$year, filter(models_1940[[i]]$x, rep(1/15,15), sides = 2), col = "forestgreen", lwd = 2)

    }))
}; dev.off()

png 
  2

In [12]:
prep_window(c(4,4))

jpeg("fig/expl-sa_era5_cdsr-gmsttrend_start-sensitivity.jpg", width = 480 * 3, height = 480 * 2); par(mfrow = c(4,4), lwd = 2, cex = 1.2, pch = 20, mar = c(3,3,2,1)); {
    invisible(sapply(1:16, function(i) {    
        plot(df$gmst, df[,-(1:2)][,i], main = models_1940[[i]]$varnm, ylab = models_1940[[i]]$varnm)

        lines(models_1940[[i]]$data$gmst, ns_pars(models_1940[[i]])$loc)
        lines(models_1950[[i]]$data$gmst, ns_pars(models_1950[[i]])$loc, col = "darkblue")
        lines(models_1960[[i]]$data$gmst, ns_pars(models_1960[[i]])$loc, col = "darkred")
        
        lines(df$gmst, eff_return_level(6, models_1940[[i]], fixed_cov = df$gmst))
        lines(df$gmst, eff_return_level(6, models_1950[[i]], fixed_cov = df$gmst), col = "darkblue")
        lines(df$gmst, eff_return_level(6, models_1960[[i]], fixed_cov = df$gmst), col = "darkred")
        
        lines(df$gmst[order(df$gmst)], filter(models_1940[[i]]$x[order(df$gmst)], rep(1/15,15), sides = 2), col = "forestgreen", lwd = 2)
        points(df[df$year == 2023, c("gmst", models_1940[[i]]$varnm)], pch = 0, col = "magenta", lwd = 2)

    }))
}; dev.off()

png 
  2